In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "4,5"

In [2]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2LMHeadModel, AutoTokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
from tqdm.notebook import tqdm
import pandas as pd

2024-07-31 00:37:29.566836: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-31 00:37:30.196605: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [16]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2',pad_token='<pad>')


In [3]:
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2',pad_token='<pad>')
#special_tokens_dict = {'additional_special_tokens': ['<usr>','<sys>']}
#num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
#tokenizer.pad_token_id = tokenizer.eos_token_id
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')
model.resize_token_embeddings(len(tokenizer))
model.config.attn_pdrop = 0.05  # Dropout rate for attention probabilities
model.config.resid_pdrop = 0.05  # Dropout rate for hidden states
model.config.embd_pdrop = 0.05

In [5]:
# Load and preprocess your dataset
dataset = load_dataset('csv',
                       data_files = {'train': "./dataset/train_freeform_de.tsv",
                                     'test': "./dataset/test_freeform_de.tsv"}, 
                       delimiter='\t')

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [6]:


def preprocess_function(examples):

    inputs = examples['input_sequence']
    targets = examples['text_label']
    
    # Combine input and output with a separator
    combined_text = [f"{input_seq} <SEP> {text_label}" for input_seq, text_label in zip(inputs, targets)]
    
    # Tokenize combined text
    model_inputs = tokenizer(combined_text, max_length=32, truncation=True, padding='max_length')
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=10, truncation=True, padding='max_length')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    save_steps=5000,
    logging_steps=2500,
    save_total_limit=2,
)

# Data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    data_collator=data_collator,
)

# Train the model


Map:   0%|          | 0/943770 [00:00<?, ? examples/s]

/home/sslunder24/env/transformer/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/95780 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [13]:
num = 1213
print(tokenizer.decode(tokenized_datasets['train'][num]['input_ids']).replace('<pad>', ''))
print(tokenizer.decode(tokenized_datasets['train'][num]['labels']).replace('<pad>', ''))

새 집을 사는 것에 흥분하고 샌디와 론이 새 집을 샀을 때 <SEP> 합리적이에요.
합리적이에요.


In [14]:
trainer.train()

/home/sslunder24/env/transformer/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
2500,2.941900
5000,2.643700
7500,2.539300
10000,2.395600
12500,2.355700
15000,2.311700
17500,2.210000
20000,2.194700


/home/sslunder24/env/transformer/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sslunder24/env/transformer/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sslunder24/env/transformer/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/sslunder24/env/transformer/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was a

TrainOutput(global_step=22122, training_loss=2.4232889673246656, metrics={'train_runtime': 8828.1436, 'train_samples_per_second': 320.714, 'train_steps_per_second': 2.506, 'total_flos': 4.623742144512e+16, 'train_loss': 2.4232889673246656, 'epoch': 3.0})

In [17]:
#model.save_pretrained('./output/kogpt2delphi/')
tokenizer.save_pretrained('./output/kogpt2delphi/')

('./output/kogpt2delphi/tokenizer_config.json',
 './output/kogpt2delphi/special_tokens_map.json',
 './output/kogpt2delphi/vocab.json',
 './output/kogpt2delphi/merges.txt',
 './output/kogpt2delphi/added_tokens.json',
 './output/kogpt2delphi/tokenizer.json')